In [ ]:
!pip install -qU datasets pod-gpt pinecone-client[grpc] langchain openai tqdm feedparser pandas

In [ ]:
OPENAI_API_KEY = "enter-open-AI-key-here"
PINECONE_API_KEY = "enter-pinecone-api-key-here"
PINECONE_ENV = "us-west1-gcp-free"
#PINECONE_API_KEY = "0c3a8fb5-1eda-4ddb-a9f4-49a65218bcd1"
#PINECONE_ENV = "asia-southeast1-gcp-free"

In [ ]:
# prompt: connect to google drives and open the file newsroomrobots.csv

#from google.colab import drive
#drive.mount('/content/drive')


In [ ]:
import pandas as pd


# Create a path to the file
file_path = '/content/newsroomrobots_dataset.csv'

# Read the file into a DataFrame
df = pd.read_csv(file_path)

# Print the head of the DataFrame
print(df.head())

# List out the names of the columns
print(df.columns)


        channel_id  video_id                      published  \
0  newsroom_robots        20  Wed, 16 Aug 2023 23:56:27 GMT   
1  newsroom_robots        19  Wed, 09 Aug 2023 23:56:27 GMT   
2  newsroom_robots        18  Thu, 03 Aug 2023 18:12:05 GMT   
3  newsroom_robots        17  Wed, 02 Aug 2023 16:55:13 GMT   
4  newsroom_robots        16  Thu, 27 Jul 2023 02:08:24 GMT   

                                               title  \
0  Jeremy Caplan: Exploring the Landscape of AI T...   
1  Scott Brodbeck: Early Experiments with Generat...   
2  Elite Truong: From The Washington Post to Amer...   
3  Elite Truong: From The Washington Post to Amer...   
4  Aimee Rinehart: How AI is Transforming News Pr...   

                                              source  \
0  https://shows.acast.com/newsroomrobots/episode...   
1  https://shows.acast.com/newsroomrobots/episode...   
2  https://shows.acast.com/newsroomrobots/episode...   
3  https://shows.acast.com/newsroomrobots/episode...   
4  h

In [ ]:
# def split_transcript(speaker_text, channel_id, video_id, published, title, source, description):
#     new_rows = []
#     current_speaker = ''
#     current_segment = ''

#     for line in speaker_text.split('\n'):
#         if not line.strip():  # Skip empty lines
#             continue

#         # Split each line by timestamp
#         parts = line.split('  ')
#         if len(parts) == 2:  # This line contains a speaker name and a timestamp
#             speaker, text = parts

#             if current_speaker:  # If not the first speaker, save the segment
#                 new_rows.append([channel_id, video_id, published, title, source, description, f"{current_speaker} says \"{current_segment.strip()}\""])

#             current_speaker = speaker
#             current_segment = text.strip()  # Initialize a new segment
#         else:
#             # This line is a continuation of the last speaker's text
#             current_segment += ' ' + line.strip()

#         # Check if we've reached 300 words
#         if len(current_segment.split()) >= 300:
#             new_rows.append([channel_id, video_id, published, title, source, description, f"{current_speaker} says \"{current_segment.strip()}\""])
#             current_segment = ''

#     if current_segment:
#         new_rows.append([channel_id, video_id, published, title, source, description, f"{current_speaker} says \"{current_segment.strip()}\""])

#     return new_rows

# # Assume df is your DataFrame
# df = data

# new_rows_list = []

# for index, row in df.iterrows():
#     new_rows = split_transcript(row['transcript'], row['channel_id'], row['video_id'], row['published'], row['title'], row['source'], row['description'])
#     new_rows_list.extend(new_rows)

# new_df = pd.DataFrame(new_rows_list, columns=['channel_id', 'video_id', 'published', 'title', 'source', 'description', 'transcript'])"


In [ ]:
# new_df.head()

In [ ]:
import openai
openai.api_key = "enter-open-ai-key-here"
openai.Engine.list()  # check we have authenticated
MODEL = "text-embedding-ada-002"

In [ ]:
import pinecone

pinecone.init(
    api_key=PINECONE_API_KEY,  # app.pinecone.io
    environment=PINECONE_ENV  # next to API key in console
)

INDEX_NAME = "pod-gpt"
DIMENSIONALITY = 1536

index = pinecone.Index(INDEX_NAME)

pinecone.delete_index(INDEX_NAME)
# index.delete(delete_all=True, namespace='pod-gpt')

pinecone.create_index("pod-gpt", dimension=DIMENSIONALITY)


In [ ]:
# Initialize empty lists to hold extracted and processed values

# Loop through the 'transcript' column and extract speaker, timestamp, and text
for _, irow in df.iterrows():
    row = irow['transcript']
    print(f"row:{row}")
    new_speaker = []
    new_timestamp = []
    new_text = []

    fragments = row.split("\n\n")
    print(fragments)
    print(len(fragments))
    print(f"fragments[0]{fragments[0]}")
    print(f"fragments[1]{fragments[1]}")
    for fragment in fragments:
        parts = fragment.split('\n')
        if len(parts) >= 2:
            print(f"parts[0]:{parts[0]}")
            speaker, timestamp = parts[0].strip().split("  ")
            # speaker = parts[0].strip()
            # timestamp = parts[1].strip()
            print(f"speaker: {speaker}")
            print(f"timestamp: {timestamp}")
            text = parts[1].strip()

            # Break text into 300-word chunks
            words = text.split('/n')
            for i in range(0, len(words), 300):
                chunk = ' '.join(words[i:i+300])
                new_text.append(f'{speaker} says "{chunk}"')
                print("adding new_speaker")
                new_speaker.append(speaker)
                new_timestamp.append(timestamp)

    res = openai.Embedding.create(input=new_text, engine=MODEL)

    embeds = [record['embedding'] for record in res['data']]
    print("embeds dimensionality:")
    print(len(embeds))
    print(len(embeds[0]))

    with_md = []
    i = 0
    for text, speaker, timestamp in zip(new_text, new_speaker, new_timestamp):
    #     with_md.append(("name", vector, {"key": "value,.."}))
        with_md.append((f"{irow['video_id']}_{timestamp}", embeds[i], {"episode_number": str(irow['video_id']), "title": irow['title'], "description":irow['description'], "meta_source":irow['source'], "speaker": speaker, "timestamp": timestamp, "text_2":text}))
        i = i+1

    print(f"with_md[0]: {with_md[0]}")
    index.upsert(with_md)

# print("new_speaker_len")
# len(new_speaker)

# for index, row in df.iterrows():
#     new_rows = split_transcript(row['transcript'], row['channel_id'], row['video_id'], row['published'], row['title'], row['source'], row['description'])
#     new_rows_list.extend(new_rows)

# new_df = pd.DataFrame(new_rows_list, columns=['channel_id', 'video_id', 'published', 'title', 'source', 'description', 'transcript'])

# Create new DataFrame with processed value

# new_df = pd.DataFrame({
#     'speaker': new_speaker,
#     'timestamp': new_timestamp,
#     'processed_transcript': new_text
# })

# # print(new_df)
# print("New_def_shape")
# new_df.shape

row:Nikita Roy  0:02  
This is newsroom robots, the Podcast where we explore the intersection of artificial intelligence and the news industry. I'm Nikita Roy, a data scientist, media entrepreneur, and one of the many founders currently building their ventures at the Harvard innovation labs. On the newsroom robots. I'm excited to bring you insightful conversations with industry experts about how AI is impacting the way we do journalism. Today, I'm joined by Jeremy Kaplan, who serves as the Director of Teaching and Learning at the Newmark Graduate School of Journalism at the City University of New York. He also leads the school's entrepreneurial journalism creators program 100 day online curriculum that guides independent journalists to build new ventures around the world. before transitioning to academia, Jeremy was a reporter at Time Magazine, where he wrote about digital innovation. Now he's the voice behind wonder tools, a weekly newsletter that zeroes in on the most useful digital 

In [ ]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 576}},
 'total_vector_count': 576}

Reformat into format required by pod-gpt indexer

In [ ]:
# prompt: show the length of new_speaker

print("new_speaker_len")
len(new_speaker)


new_speaker_len


17

In [ ]:
# import pod_gpt

# indexer = pod_gpt.Indexer(
#     openai_api_key=OPENAI_API_KEY,
#     pinecone_api_key=PINECONE_API_KEY,
#     pinecone_environment=PINECONE_ENV,
#     index_name="pod-gpt"
# )


In [ ]:
# for index, row in data.iterrows():
#     print(row)
#     row_dict = row.to_dict()
    # indexer(pod_gpt.VideoRecord(**row_dict))

In [ ]:
for index, row in data.iterrows():
    print(row)

NameError: ignored

In [ ]:

import pinecone

pinecone.init(
    api_key=PINECONE_API_KEY,  # app.pinecone.io
    environment=PINECONE_ENV  # next to API key in console
)

index_name = "pod-gpt"

if index_name not in pinecone.list_indexes():
    raise ValueError(
        f"No '{index_name}' index exists. You must create the index before "
        "running this notebook. Please refer to the walkthrough at "
        "'github.com/pinecone-io/examples'."  # TODO add full link
    )

index = pinecone.Index(index_name)

In [ ]:

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone

embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

# embeddings.embed_query

vectordb = Pinecone(
    index=index,
    embedding_function=embeddings.embed_query,
    text_key="text"
)

In [ ]:
from langchain.chat_models import ChatOpenAI

llm=ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    temperature=0,
    model_name='gpt-3.5-turbo'
)


In [ ]:
from langchain.chains import RetrievalQA

retriever = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectordb.as_retriever()
)


In [ ]:

tool_desc = """Use this tool to answer user questions using Lex
Fridman podcasts. If the user states 'ask Lex' use this tool to get
the answer. This tool can also be used for follow up questions from
the user."""


In [ ]:

from langchain.agents import Tool

tools = [Tool(
    func=retriever.run,
    description=tool_desc,
    name='Lex Fridman DB'
)]

In [ ]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    memory_key="chat_history",  # important to align with agent prompt (below)
    k=5,
    return_messages=True
)


In [ ]:
from langchain.agents import initialize_agent

conversational_agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=2,
    early_stopping_method="generate",
    memory=memory,
)


In [ ]:
conversational_agent.agent.llm_chain.prompt


In [ ]:

sys_msg = """You are a helpful chatbot that answers the user's questions.
"""

prompt = conversational_agent.agent.create_prompt(
    system_message=sys_msg,
    tools=tools
)
conversational_agent.agent.llm_chain.prompt = prompt

In [ ]:
conversational_agent.agent.llm_chain.prompt


In [ ]:
conversational_agent.agent.llm_chain.prompt.input_variables


In [ ]:
conversational_agent.agent.llm_chain.prompt.messages


In [ ]:
conversational_agent.agent.llm_chain.prompt.messages[0]


In [ ]:

print(
    conversational_agent.agent.llm_chain.prompt.messages[0].prompt.template
)


In [ ]:
conversational_agent.agent.llm_chain.prompt.messages[1]


In [ ]:
conversational_agent.agent.llm_chain.prompt.messages[2]


In [ ]:

print(
    conversational_agent.agent.llm_chain.prompt.messages[2].prompt.template
)

In [ ]:
conversational_agent("hi how are you")


In [ ]:
conversational_agent("what are the top episodes of lex's podcast")


In [ ]:
!pip install feedparser


In [ ]:
# prompt: write the code to get all the data that gets sent back from this rss url -- RSS_URL = "https://feeds.acast.com/public/shows/64c86a5585617f0011a4a263"

import feedparser

RSS_URL = "https://feeds.acast.com/public/shows/64c86a5585617f0011a4a263"

feed = feedparser.parse(RSS_URL)

for entry in feed.entries:
    print(entry.title)
    print(entry.link)
    print(entry.description)
    print(entry.published)
    print(entry.itunes_episode)




In [ ]:
import feedparser
import pandas as pd

RSS_URL = "https://feeds.acast.com/public/shows/64c86a5585617f0011a4a263"

feed = feedparser.parse(RSS_URL)

# Create lists to store the details
titles = []
links = []
descriptions = []
published_dates = []
itunes_episodes = []

for entry in feed.entries:
    titles.append(entry.title)
    links.append(entry.link)
    descriptions.append(entry.description)
    published_dates.append(entry.published)

    # Check for the specific title and modify itunes_episode accordingly
    if entry.title == 'Introducing: Newsroom Robots with Nikita Roy':
        itunes_episodes.append(0)
    else:
        itunes_episodes.append(getattr(entry, 'itunes_episode', None))  # Use getattr to avoid KeyError

# Create a DataFrame from the gathered details
df = pd.DataFrame({
    'iTunes Episode': itunes_episodes,
    'Published Date': published_dates,
    'Title': titles,
    'Link': links,
    'Description': descriptions
})

# Display the DataFrame (optional)
print(df)


In [ ]:
from google.colab import drive

# Mount the drive
drive.mount('/content/drive')

In [ ]:
import os
import pandas as pd

# Create a path to the folder
folder_path = '/content/drive/My Drive/transcribed_texts'

# List the files in the folder
files = os.listdir(folder_path)

# Assuming df is your DataFrame
transcripts = []

# Open the files in the folder and read contents
for file in files:
    with open(os.path.join(folder_path, file), 'r') as f:
        transcripts.append(f.read())

# Add the content to the DataFrame
df['transcript'] = transcripts


In [ ]:
# prompt: give me the the whole cell in transcript in the first row

print(df['transcript'][1])

In [ ]:
import feedparser

RSS_URL = "https://feeds.acast.com/public/shows/64c86a5585617f0011a4a263"

def print_recursive(data, indent=0):
    """Recursively print all items in the data structure."""

    # Check if the data is a dictionary
    if isinstance(data, dict):
        for key, value in data.items():
            print('  ' * indent + f"{key}:")
            print_recursive(value, indent + 1)
    # Check if the data is a list
    elif isinstance(data, list):
        for index, item in enumerate(data):
            print('  ' * indent + f"Item {index}:")
            print_recursive(item, indent + 1)
    # If the data is neither a dictionary nor a list, print it directly
    else:
        print('  ' * indent + f"{data}")

def get_rss_data(url):
    """Fetch and print data from the given RSS URL."""
    # Parse the RSS feed
    data = feedparser.parse(url)

    # Print all details
    print_recursive(data)

if __name__ == "__main__":
    get_rss_data(RSS_URL)


In [ ]:
# prompt: open the file newsroomrobots.csv from google drive, load it as a dataframe using pandas and then print the head


# Create a path to the file
file_path = '/content/drive/My Drive/newsroomrobots.csv'

# Read the file into a DataFrame
df = pd.read_csv(file_path)

# Print the head of the DataFrame
print(df.head())


In [ ]:

# Create a path to the folder
folder_path = '/content/drive/My Drive/transcribed_texts'

# List the files in the folder
files = os.listdir(folder_path)

# Add a 'transcript' column with default empty strings if it doesn't exist
if 'transcript' not in df.columns:
    df['transcript'] = ''

# Open the files in the folder and read contents
for file in files:
    # Extract title from file name
    title_from_file = file.split('-')[1:]
    title_from_file = ': '.join([title_from_file[0], ' '.join(title_from_file[1:])])

    # Check if the title exists in the DataFrame using the "Title" column
    mask = df['Title'] == title_from_file

    if mask.any():
        with open(os.path.join(folder_path, file), 'r') as f:
            df.loc[mask, 'transcript'] = f.read()


In [ ]:
# prompt: print the first line of the df and print the entire text in the cell

print(df.head(1))
print(df['transcript'][1])


In [ ]:
df.head(1)

In [ ]:
print(df['transcript'][1])

In [ ]:
print(df.head())

In [ ]:
# prompt: save df to google drive as a file called newsroomrobots.csv

df.to_csv('/content/drive/My Drive/newsroomrobots.csv')
